In [1]:
from dipy.io.stateful_tractogram import StatefulTractogram
from dipy.io.streamline import load_tractogram
import taubrain as tb
import pandas as pd
tracto = load_tractogram('data/data.trk', 'same', bbox_valid_check=False)
streamlines = tracto.streamlines
tau=tb.tortuosity.tortuosity_geometric(streamlines)
df=pd.read_csv("steam_tg.csv")
df

Results
 tortuosity:3.3229331510799396


,stream_length,tau
0,14.0,1.004970
1,13.0,1.004880
2,19.0,1.004344
3,17.0,1.003831
4,338.0,2.725962
...,...,...
2904,153.0,3.175735
2905,181.0,5.207267
2906,178.0,5.366423
2907,157.0,3.579689
